In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.text("catalogo", "credit_catalog")
dbutils.widgets.text("esquema_source", "bronze")
dbutils.widgets.text("esquema_sink", "silver")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")

In [0]:
df_credit_record = spark.table(f"{catalogo}.{esquema_source}.credit_record")

In [0]:
df_credit_record = df_credit_record.dropna(how="all")\
                    .filter((col("ID").isNotNull()) | (col("STATUS")).isNotNull())

In [0]:
df_credit_record_clean = df_credit_record.withColumn(
    "is_overdue",
    when(col("STATUS").isin("1","2","3","4","5"),1).otherwise(0)
)

In [0]:
df_credit_record_clean.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.credit_record_transformed")